In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models_torch import *
import torch.optim as optim
from random import shuffle

In [3]:
train_tape = "Z:/rppg/ubfc_datatape_160x128x128_train.h5"
valid_tape = "Z:/rppg/ubfc_datatape_160x128x128_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def RandomHorizontalFlip(data, label, p=.5):
    if np.random.random()<p:
        return data[..., ::-1 ,:], label
    return data, label

def data_loader(tape, batch_size=4, shuffle_size=2000):
    shuffle_size = max(batch_size, shuffle_size)
    buff = []
    for i in tape:
        i = [i.astype(np.float16) for i in i]
        if not 40<get_hr(i[1])<179:
            continue
        buff.append(RandomHorizontalFlip(*i))
        if len(buff)>=shuffle_size:
            shuffle(buff)
            batch = [buff.pop(0) for _ in range(batch_size)]
            yield torch.tensor(np.array([i[0][..., [2, 1, 0]].transpose(3, 0, 1, 2) for i in batch])).float()*2-1, torch.tensor(np.array([i[1] for i in batch]))
    while len(buff)>=batch_size:
        shuffle(buff)
        batch = [buff.pop(0) for _ in range(batch_size)]
        yield torch.tensor(np.array([i[0][..., [2, 1, 0]].transpose(3, 0, 1, 2) for i in batch])).float()*2-1, torch.tensor(np.array([i[1] for i in batch]))

In [4]:
gra_sharp = 2.
model = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160,128,128), patches=(4,4,4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7).cuda()
lr=0.0001
step_size = 50
batch_size=4
optimizer1 = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00005)
scheduler1 = optim.lr_scheduler.StepLR(optimizer1, step_size=step_size, gamma=0.5)

criterion_Pearson = Neg_Pearson()
valid_baches = 1000

# a --> Pearson loss; b --> frequency loss
a_start = 0.1
b_start = 1
exp_b = 5

In [4]:
best = np.inf
for epoch in range(10):
    loss_rPPG_avg = []
    loss_peak_avg = []
    loss_kl_avg_test = []
    loss_hr_mae = []
    model.train()
    fps = 30
    print(f'epoch:{epoch} train:')
    for i, (data, label) in enumerate(data_loader(train, batch_size=batch_size)):
        hr = torch.tensor([get_hr(i) for i in label]).float().cuda()
        data, label = data.float().cuda(), label.float().cuda()
        optimizer1.zero_grad()
        bvp, s1, s2, s3 = model(data, gra_sharp)
        bvp = (bvp-torch.mean(bvp, axis=-1).view(-1, 1))/torch.std(bvp, axis=-1).view(-1, 1)
        loss_bvp = criterion_Pearson(bvp, label)
        fre_loss = .0
        kl = .0
        train_mae = .0
        for bb in range(data.shape[0]):
            loss_distribution_kl, fre_loss_temp, train_mae_temp = cross_entropy_power_spectrum_DLDL_softmax2(bvp[bb], hr[bb], fps, std=1.0)
            fre_loss = fre_loss+fre_loss_temp
            kl = kl+loss_distribution_kl
            train_mae = train_mae+train_mae_temp
        fre_loss /= data.shape[0]
        kl /= data.shape[0]
        train_mae /= data.shape[0]
        a = a_start
        if epoch>10:
            b = b_start*exp_b
        else:
            b = b_start*exp_b**(epoch/10)
        loss = a*loss_bvp + b*(fre_loss+kl)
        loss.backward()
        optimizer1.step()
        n = data.size(0)
        loss_rPPG_avg.append(float(loss_bvp.data))
        loss_peak_avg.append(float(fre_loss.data))
        loss_kl_avg_test.append(float(kl.data))
        loss_hr_mae.append(float(train_mae))
        print('epoch:%d, batch:%d, total=%d, lr=%f, sharp=%.3f, a=%.3f, NegPearson= %.4f, b=%.3f, kl= %.4f, fre_CEloss= %.4f, hr_mae= %.4f' % (epoch, i + 1, len(train)//batch_size, lr, gra_sharp, a, np.mean(loss_rPPG_avg[-2000:]), b, np.mean(loss_kl_avg_test[-2000:]), np.mean(loss_peak_avg[-2000:]), np.mean(loss_hr_mae[-2000:])), end='\r')
    optimizer1.zero_grad()
    model.eval()
    with torch.no_grad():
        hrs = []
        for j, (data, label) in enumerate(data_loader(valid, batch_size=4)):
            data, label = data.cuda().float(), label.cuda().float()
            bvp, s1, s2, s3 = model(data, gra_sharp)
            bvp = (bvp-torch.mean(bvp, axis=-1).view(-1, 1))/torch.std(bvp).view(-1, 1)
            for _1, _2 in zip(bvp, label):
                hrs.append((get_hr(_1.cpu().detach().numpy()), get_hr(_2.cpu().detach().numpy())))
        RMSE = np.mean([(i-j)**2 for i, j in hrs])**0.5
        print(f'Test RMSE:{RMSE:.3f}, batch:{i+1}')
        if RMSE<best:
            best = RMSE
            torch.save(model.state_dict(), '../weights/Physformer_UBFC.pkl')
            print('Best model saved')
        scheduler1.step()
        if (epoch + 1) % step_size == 0:
            lr *= 0.5

epoch:0 train:
Test RMSE:9.892, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.9782, b=1.000, kl= 3.5053, fre_CEloss= 4.9240, hr_mae= 16.7771
Best model saved
epoch:1 train:
Test RMSE:4.676, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.7009, b=1.175, kl= 3.4827, fre_CEloss= 4.9012, hr_mae= 2.4957
Best model saved
epoch:2 train:
Test RMSE:4.051, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.6070, b=1.380, kl= 3.4798, fre_CEloss= 4.8983, hr_mae= 1.8244
Best model saved
epoch:3 train:
Test RMSE:2.812, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.5586, b=1.621, kl= 3.4785, fre_CEloss= 4.8970, hr_mae= 1.5864
Best model saved
epoch:4 train:
Test RMSE:4.330, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.5238, b=1.904, kl= 3.4778, fre_CEloss= 4.8963, hr_mae= 1.6439
epoch:5 train:
Test RMSE:2.665, batch:121121, lr=0.000100, sharp=2.000, a=0.100, NegPearson= 0.4934, b=2.236, kl= 3.4772, fre_CEloss= 4.8957, hr_m

In [5]:
model.load_state_dict(torch.load('../weights/Physformer_UBFC.pkl'))
model.train()
def physformer(x, model=model):
    with torch.no_grad():
        bvp = model(torch.tensor(x[..., [2, 1, 0]].transpose(0, 4, 1, 2, 3)).cuda().float()*2-1, gra_sharp)[0].cpu().detach().numpy()
        #bvp[:,:30] = bvp[:,130:] = bvp[:, 30:130].mean(axis=-1).reshape(-1, 1)
        return bvp - bvp.mean(axis=-1).reshape(-1, 1)

In [6]:
eval_on_dataset(test_set_PURE, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_UBFC_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_UBFC_PURE.h5')

100%|██████████| 59/59 [05:00<00:00,  5.09s/it]


{'Sliding window': {'MAE': 18.504, 'RMSE': 30.313, 'R': 0.34842},
 'Whole video': {'MAE': 17.603, 'RMSE': 29.024, 'R': 0.39774}}